##### Investment Strategy

In [1]:
import pandas as pd
import numpy as np
import time
import statsmodels.api as sm
from tqdm import tqdm
import os


path_data = '/Users/hongt/Desktop/UBC MBAN/P4/BAFI 508/PJ/Final/'



In [2]:
os.listdir(path_data)

['.ipynb_checkpoints',
 'CRSP_Dividend.ipynb',
 'dividend_yearly.csv',
 'F-F_Research_Data_Factors_2018.csv',
 'ikd7pnvlgcwtgfuu.csv',
 'InvestmentStrategy_Dividend_yield02.ipynb',
 'portfolios2.xlsx']

## Step 1: Preparing the CRSP file

In [3]:
print("Prepare CRSP file")
t = time.time() # record the current time, so we can measure how long the code takes to run

crsp = pd.read_csv(path_data+'ikd7pnvlgcwtgfuu.csv')
print('Completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block

Prepare CRSP file
Completed in 3.5s


In [4]:
crsp.head(10)

,PERMNO,date,SHRCD,EXCHCD,TICKER,COMNAM,PERMCO,PRC,RET,SHROUT,RETX
0,10000,1985-12-31,NaN,NaN,NaN,NaN,7952,NaN,NaN,NaN,NaN
1,10000,1986-01-31,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-4.37500,C,3680.0,C
2,10000,1986-02-28,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-3.25000,-0.257143,3680.0,-0.257143
3,10000,1986-03-31,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-4.43750,0.365385,3680.0,0.365385
4,10000,1986-04-30,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-4.00000,-0.098592,3793.0,-0.098592
5,10000,1986-05-30,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-3.10938,-0.222656,3793.0,-0.222656
6,10000,1986-06-30,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-3.09375,-0.005025,3793.0,-0.005025
7,10000,1986-07-31,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-2.84375,-0.080808,3793.0,-0.080808
8,10000,1986-08-29,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-1.09375,-0.615385,3793.0,-0.615385
9,10000,1986-09-30,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-1.03125,-0.057143,3793.0,-0.057143


In [5]:
sum(crsp['RET'].isna())

34046

In [6]:
crsp.dtypes

PERMNO      int64
date       object
SHRCD     float64
EXCHCD    float64
TICKER     object
COMNAM     object
PERMCO      int64
PRC       float64
RET        object
SHROUT    float64
RETX       object
dtype: object

#### Formatting Data:

In [7]:
### formatting ###
# make all variable names lowercase
t = time.time()
crsp.columns = map(str.lower,crsp.columns)

# Changes the returns to number format. Non-numeric data will be NAN
crsp['ret'] = pd.to_numeric(crsp['ret'],errors='coerce') 

# Change the dateformat
crsp['date'] = pd.to_datetime(crsp['date'], format='%Y-%m-%d')

# Create separate 'year' and 'month' variables
crsp['year'] = crsp['date'].apply(lambda date: date.year)
crsp['month'] = crsp['date'].apply(lambda date: date.month)

# Calculate market cap
crsp['mktcap'] = crsp['shrout'] * crsp['prc'].abs()


### Some basic data cleaning ###
# keep only common shares
crsp = crsp[crsp['shrcd'].isin([10,11])]

# keep only stocks from NYSE, AMEX and NASDAQ
crsp = crsp[crsp['exchcd'].isin([1,2,3])]

# make sure that there are no duplicates
# usually, we would investigate why there are duplicates and then decide which observation we want to keep
#    For here, it is enough to simply drop the duplicates.
crsp = crsp.drop_duplicates(subset=['date','permno'])


print('Completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block


Completed in 20.6s


### Step 2: Loading the Dividened_Yield File (Already prepared in another notebook)

In [8]:
print("Prepare dividend_yield file")
t = time.time() 
dv = pd.read_csv(path_data+'dividend_yearly.csv')
print('Completed in %.1fs' % (time.time()-t))

Prepare dividend_yield file
Completed in 0.7s


In [9]:
dv=dv.drop('Unnamed: 0',axis=1)
dv.head()

,PERMNO,SHRCD,EXCHCD,TICKER,COMNAM,PERMCO,PRC,SHROUT,date,RET_DM,dvy
0,10000,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-3.09375,3793.0,1986-06-30,0.000000,0.000000
1,10000,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,NaN,3893.0,1987-06-30,0.000000,NaN
2,10001,11.0,3.0,GFGC,GREAT FALLS GAS CO,7953,-6.12500,985.0,1986-06-30,0.031834,192.058501
3,10001,11.0,3.0,GFGC,GREAT FALLS GAS CO,7953,5.87500,991.0,1987-06-30,0.066416,386.682254
4,10001,11.0,3.0,GFGC,GREAT FALLS GAS CO,7953,6.25000,992.0,1988-06-30,0.066508,412.349600


In [10]:
sum(dv['dvy'].isna())

3160

We get rid of the NaN values for dividend yield.

In [11]:
dv=dv[~dv['dvy'].isna()]

In [12]:
sum(dv['dvy']==0)/len(dv)

0.5155702911109437

51% of the rows have zero dividend yield. We will put them into a separate portfolio and leave those with positive dividend yield to build five portfolios.

In [13]:
dv_0=dv[dv['dvy']==0].reset_index(drop=True)

In [14]:
dv=dv[dv['dvy']>0].reset_index(drop=True)

#### Formatting:

In [15]:
#For dv dataframe
t = time.time() 

dv.columns = map(str.lower,dv.columns)

# Change the dateformat 

dv['date'] = pd.to_datetime(dv['date'])


# Create separate 'year' and 'month' variables
dv['year'] = dv['date'].apply(lambda x: x.year)
dv['month'] = dv['date'].apply(lambda x: x.month)


### Drop duplicates ###
#ccm = ccm.drop_duplicates(subset=['datadate','gvkey'])
#ccm = ccm.drop_duplicates(subset=['year','gvkey'])
#ccm = ccm.drop_duplicates(subset=['year','lpermno'])

print('Completed in %.1fs' % (time.time()-t))

Completed in 0.4s


In [16]:
#For dv_0 dataframe
t = time.time() 

dv_0.columns = map(str.lower,dv_0.columns)

# Change the dateformat 

dv_0['date'] = pd.to_datetime(dv_0['date'])


# Create separate 'year' and 'month' variables
dv_0['year'] = dv_0['date'].apply(lambda x: x.year)
dv_0['month'] = dv_0['date'].apply(lambda x: x.month)


### Drop duplicates ###
#ccm = ccm.drop_duplicates(subset=['datadate','gvkey'])
#ccm = ccm.drop_duplicates(subset=['year','gvkey'])
#ccm = ccm.drop_duplicates(subset=['year','lpermno'])

print('Completed in %.1fs' % (time.time()-t))

Completed in 0.4s


### Step 3: Sort stocks into portfolios and calculate returns

In [17]:
len(dv['permno'].unique())

13805

In [18]:
len(list(dv[dv['year']==1996]['permno'].unique()))

0

In [19]:
dv_0.head()

,permno,shrcd,exchcd,ticker,comnam,permco,prc,shrout,date,ret_dm,dvy,year,month
0,10000,10.0,3.0,OMFGA,OPTIMUM MANUFACTURING INC,7952,-3.09375,3793.0,1986-06-30,0.0,0.0,1986,6
1,10001,11.0,3.0,EWST,ENERGY WEST INC,7953,6.66000,2599.0,2004-06-30,0.0,0.0,2004,6
2,10001,11.0,3.0,EWST,ENERGY WEST INC,7953,9.05000,2913.0,2005-06-30,0.0,0.0,2005,6
3,10002,10.0,3.0,MBNC,MOBILE NATIONAL CORP,7954,-12.68750,1175.0,1986-06-30,0.0,0.0,1986,6
4,10002,10.0,3.0,MBNC,MOBILE NATIONAL CORP,7954,-13.50000,1175.0,1987-06-30,0.0,0.0,1987,6


In [20]:
permno_list=[]
for year in tqdm(range(1986,2017),desc="years"):
    # take the companies that were alive at t-1
    permno_list=list(dv[dv['year']==year-1]['permno'].unique())
    
len(permno_list)

years: 100%|██████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 178.13it/s]


4359

In [21]:
#Create 5 portfolios based on dv dataframe (companies with positive dividend yield)
print("Create portfolios")
t = time.time() # reset our timer

# loop over all years in the data
# Note: the first loop loops over the years in range(1981,2017).

portfolios = [] # create an empty list to collect the portfolio returns
for year in tqdm(range(1986,2017),desc="years"):
    # take the companies that were alive at t-1
    permno_list=list(dv[dv['year']==year-1]['permno'].unique()) 
    
    # get the sorting variable for these companies at t-1
    sorting_data = dv.loc[(dv['year']==(year-1)) & \
                           (dv['permno'].isin(permno_list)), \
                           ['permno','permco','dvy']]
    
    if(len(sorting_data)==0):
        continue
    
    # sort into 5 baskets by cashflow over assets
    nportfolios = 5 # number of portfolios
    sorting_data['rank'] = pd.qcut(sorting_data['dvy'],nportfolios, labels=False, precision=3)
    
    # select the return data with some time lag to make sure that the accounting information is public (data from July at year t to June in year t+1)
    crsp_window = crsp[((crsp['year']==year) & (crsp['month']>=6)) | \
                       ((crsp['year']==year+1) & (crsp['month']<=6))]
    
    # create the portfolio returns for the current window and collect them in portfolios_window
    portfolios_window = [] 
    for p in range(nportfolios):
        # get list of permnos that are in this portfolio
        basket = sorting_data.loc[sorting_data['rank'] == p,'permno'].tolist()
        
        # get returns of these permnos
        crsp_p_firms = crsp_window[crsp_window['permno'].isin(basket)]
        
        # pivot returns
        returns = crsp_p_firms.pivot(index='date', columns='permno', values='ret')
        returns = returns.iloc[1:,:] # drop the first row
        
        # create equally weighted portfolio (monthly rebalancing)
        return_port = returns.mean(axis=1)
        return_port.name = str(p)
        
        # collect portfolio returns in dec_port
        portfolios_window += [return_port]
        
    # merge the portfolios
    portfolios_window = pd.concat(portfolios_window,axis=1)
        
    # collect results in portfolios
    portfolios += [portfolios_window]

# merge the returns from all windows
portfolios = pd.concat(portfolios,axis=0)


print('Step 3 completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block



Create portfolios


years: 100%|███████████████████████████████████████████████████████████████████████████| 31/31 [00:03<00:00,  8.38it/s]

Step 3 completed in 3.7s


In [22]:
portfolios

,0,1,2,3,4
date,,,,,
1987-07-31,0.026985,0.038533,0.041173,0.044731,0.037965
1987-08-31,0.023139,0.032791,0.027127,0.037693,0.034250
1987-09-30,-0.009689,-0.011472,-0.017846,-0.020380,-0.020231
1987-10-30,-0.249682,-0.254192,-0.264913,-0.242824,-0.205500
1987-11-30,-0.028631,-0.029646,-0.036449,-0.044108,-0.056027
...,...,...,...,...,...
2010-02-26,0.025364,0.027819,0.050461,0.051990,0.031755
2010-03-31,0.075771,0.084026,0.087350,0.073981,0.063763
2010-04-30,0.101874,0.108747,0.072873,0.055211,0.034885


In [23]:
portfolios.to_excel('portfolios2.xlsx')

### Step 4: Performance Evaluation
#### Step 4a: Merge Portfolio returns with Fama French data

In [24]:
### load and prepare fama french data ###
# load Fama French monthly factors
ff = pd.read_csv(path_data+'F-F_Research_Data_Factors_2018.csv')
# rename columns
ff.rename({'Mkt-RF':'ExMkt',
           'DATE':'date'},axis=1,inplace=True)

# date variables
ff['year'] = ff['date'] // 100
ff['month'] = ff['date'] % 100
ff.set_index('date',inplace=True)


### formatting ###
# FF data is in percent. Convert to simple returns
ff[['ExMkt', 'SMB', 'HML', 'RF']] /= 100


### merge portfolio returns with Fama French data ###
# date variables
portfolios_ff = portfolios.copy() # create a copy of the portfolios dataframe so we can use it again later
portfolios_ff['year'] = portfolios_ff.index.year
portfolios_ff['month'] = portfolios_ff.index.month

# merge
portfolios_ff = pd.merge(portfolios_ff,ff,on=['year','month'])


### Step 4b: Regressions

In [25]:
# show average returns (annualized and in percent)
print("Average returns (annualized percent)\n",((1+portfolios.mean(axis=0))**12-1)*100)

# Calculate the excess returns
for p in range(nportfolios):
    portfolios_ff['ExRet_'+str(p)] = portfolios_ff[str(p)]-portfolios_ff['RF']

Average returns (annualized percent)
 0    10.766450
1    10.325915
2    11.654629
3    11.575666
4    11.848978
dtype: float64


In [26]:
### Market model regressions ###
table_capm = []
for p in range(nportfolios):
    # regress portfolio excess return on market excess return
    results = sm.OLS(portfolios_ff['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff['ExMkt'])).fit()
    
    # collect results
    table_row = pd.DataFrame({'alpha':results.params['const'],
                              'beta_mkt':results.params['ExMkt'],
                              'alpha_t':results.tvalues['const'],
                              'rmse':np.sqrt(results.mse_resid),
                              'R2':results.rsquared},
                             index=[p])
    
    table_capm += [table_row]

In [27]:
# Combine the results for all portfolios
table_capm = pd.concat(table_capm,axis=0)
table_capm.index.name = 'quintile'

# show results
print("CAPM\n",table_capm)

CAPM
              alpha  beta_mkt   alpha_t      rmse        R2
quintile                                                  
0         0.002524  0.618541  1.302044  0.027591  0.501852
1         0.001780  0.730615  0.846461  0.029933  0.544252
2         0.002444  0.824540  1.184106  0.029377  0.612276
3         0.002349  0.834074  1.375685  0.024310  0.702347
4         0.002737  0.784268  1.746748  0.022304  0.712519


In [28]:
### Three Factor model regressions ###
table_ff = []
for p in range(nportfolios):
    # regress portfolio excess return on market excess return
    results = sm.OLS(portfolios_ff['ExRet_'+str(p)],
                     sm.add_constant(portfolios_ff[['ExMkt','SMB','HML']])).fit()
    
    # collect results
    table_row = pd.DataFrame({'alpha':results.params['const'],
                              'beta_mkt':results.params['ExMkt'],
                              'beta_size':results.params['SMB'],
                              'beta_hml':results.params['HML'],
                              'alpha_t':results.tvalues['const'],
                              'rmse':np.sqrt(results.mse_resid),
                              'R2':results.rsquared},
                             index=[p])
    
    table_ff += [table_row]

# Combine the results for all portfolios
table_ff = pd.concat(table_ff,axis=0)
table_ff.index.name = 'quintile'

# show results
print("Fama-French 3\n",table_ff)

Fama-French 3
              alpha  beta_mkt  beta_size  beta_hml   alpha_t      rmse  \
quintile                                                                
0         0.000245  0.683411   0.537768  0.594834  0.181858  0.018940   
1        -0.001246  0.836657   0.606683  0.787424 -1.018106  0.017236   
2        -0.000736  0.948914   0.567684  0.825861 -0.638587  0.016227   
3        -0.000330  0.974752   0.285379  0.691927 -0.293921  0.015833   
4         0.000307  0.968551  -0.046737  0.620851  0.343386  0.012584   

                R2  
quintile            
0         0.767592  
1         0.850387  
2         0.882875  
3         0.874990  
4         0.909389  


In [29]:
portfolios_ff

,0,1,2,3,4,year,month,ExMkt,SMB,HML,RF,ExRet_0,ExRet_1,ExRet_2,ExRet_3,ExRet_4
0,0.026985,0.038533,0.041173,0.044731,0.037965,1987,7,0.0385,-0.0067,0.0066,0.0046,0.022385,0.033933,0.036573,0.040131,0.033365
1,0.023139,0.032791,0.027127,0.037693,0.034250,1987,8,0.0352,-0.0072,-0.0090,0.0047,0.018439,0.028091,0.022427,0.032993,0.029550
2,-0.009689,-0.011472,-0.017846,-0.020380,-0.020231,1987,9,-0.0259,0.0052,0.0028,0.0045,-0.014189,-0.015972,-0.022346,-0.024880,-0.024731
3,-0.249682,-0.254192,-0.264913,-0.242824,-0.205500,1987,10,-0.2324,-0.0843,0.0423,0.0060,-0.255682,-0.260192,-0.270913,-0.248824,-0.211500
4,-0.028631,-0.029646,-0.036449,-0.044108,-0.056027,1987,11,-0.0777,0.0277,0.0314,0.0035,-0.032131,-0.033146,-0.039949,-0.047608,-0.059527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0.025364,0.027819,0.050461,0.051990,0.031755,2010,2,0.0340,0.0121,0.0317,0.0000,0.025364,0.027819,0.050461,0.051990,0.031755
200,0.075771,0.084026,0.087350,0.073981,0.063763,2010,3,0.0631,0.0143,0.0210,0.0001,0.075671,0.083926,0.087250,0.073881,0.063663
201,0.101874,0.108747,0.072873,0.055211,0.034885,2010,4,0.0200,0.0498,0.0281,0.0001,0.101774,0.108647,0.072773,0.055111,0.034785
202,-0.064629,-0.083818,-0.073840,-0.084716,-0.079742,2010,5,-0.0789,0.0005,-0.0238,0.0001,-0.064729,-0.083918,-0.073940,-0.084816,-0.079842
